In [ ]:
!pip uninstall -y torch-scatter torch-sparse torch-geometric
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-geometric

Found existing installation: torch-geometric 2.6.1
Uninstalling torch-geometric-2.6.1:
  Successfully uninstalled torch-geometric-2.6.1
Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 36.7 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 18.8 MB/s eta 0:00:00
  Using cached torch_geometric-2.6.1-py3-none-any.whl.metadata (63 kB)
Using cached torch_geometric-2.6.1-py3-none-any.whl (1.1 MB)


In [ ]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 606.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.1 MB/s eta 0:00:00


In [ ]:
from google.colab import runtime
def exit_on_oom():
    print('Out of memory! Restarting runtime...')
    runtime.unassign()
%set_env PYTHONFAULTHANDLER=1

env: PYTHONFAULTHANDLER=1


In [ ]:
import h5py

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, Dataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd
import gc

In [ ]:
# Конфигурация типов данных
torch.set_default_dtype(torch.float32)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Загрузка данных с контролем памяти
def load_data(path):
    df = pd.read_csv(path)
    number_cols = [
        'valence', 'year', 'acousticness', 'danceability', 'duration_ms',
        'energy', 'explicit', 'instrumentalness', 'key', 'liveness',
        'loudness', 'mode', 'popularity', 'speechiness', 'tempo'
    ]
    return df[['name', 'artists'] + number_cols].dropna().reset_index(drop=True)

data = load_data("/content/data.csv")
print(f"Loaded {len(data)} songs")

Loaded 170653 songs


In [ ]:
data

,name,artists,valence,year,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo
0,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...","['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.0594,1921,0.98200,0.279,831667,0.211,0,0.878000,10,0.6650,-20.096,1,4,0.0366,80.954
1,Clancy Lowered the Boom,['Dennis Day'],0.9630,1921,0.73200,0.819,180533,0.341,0,0.000000,7,0.1600,-12.441,1,5,0.4150,60.936
2,Gati Bali,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.0394,1921,0.96100,0.328,500062,0.166,0,0.913000,3,0.1010,-14.850,1,5,0.0339,110.339
3,Danny Boy,['Frank Parker'],0.1650,1921,0.96700,0.275,210000,0.309,0,0.000028,5,0.3810,-9.316,1,3,0.0354,100.109
4,When Irish Eyes Are Smiling,['Phil Regan'],0.2530,1921,0.95700,0.418,166693,0.193,0,0.000002,3,0.2290,-10.096,1,2,0.0380,101.665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170648,China,"['Anuel AA', 'Daddy Yankee', 'KAROL G', 'Ozuna...",0.6080,2020,0.08460,0.786,301714,0.808,0,0.000289,7,0.0822,-3.702,1,72,0.0881,105.029
170649,Halloweenie III: Seven Days,['Ashnikko'],0.7340,2020,0.20600,0.717,150654,0.753,0,0.000000,7,0.1010,-6.020,1,68,0.0605,137.936
170650,AYA,['MAMAMOO'],0.6370,2020,0.10100,0.634,211280,0.858,0,0.000009,4,0.2580,-2.226,0,76,0.0809,91.688
170651,Darkness,['Eminem'],0.1950,2020,0.00998,0.671,337147,0.623,1,0.000008,2,0.6430,-7.161,1,70,0.3080,75.055


In [ ]:
# Класс датасета с контролем типов
class TypeSafeMusicDataset(Dataset):
    def __init__(self, data, top_k=20):
        super().__init__()
        self.data = data
        self.top_k = top_k
        self.node_features, self.edge_index = self._build_graph()

    def _build_graph(self):
        # Нормализация данных
        scaler = StandardScaler()
        features = scaler.fit_transform(self.data.iloc[:, 3:].values)

        # Поиск соседей
        nbrs = NearestNeighbors(n_neighbors=self.top_k+1, metric='cosine')
        nbrs.fit(features)
        _, indices = nbrs.kneighbors(features)

        # Создание edge_index
        rows = np.repeat(np.arange(len(indices)), self.top_k)
        cols = indices[:, 1:].flatten()

        # Создание тензоров с явным указанием типа
        features_tensor = torch.tensor(features, dtype=torch.float32).to(device)
        edge_index = torch.tensor([rows, cols], dtype=torch.long).to(device)

        return features_tensor, edge_index

    def __len__(self):
        return 1

    def __getitem__(self, idx):
        return Data(x=self.node_features, edge_index=self.edge_index)

In [ ]:
class TypeSafeGCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()

        # Сохраняем параметры как атрибуты класса
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim

        # Инициализация слоев
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = x.float()
        x = F.relu(self.conv1(x, edge_index))
        return self.conv2(x, edge_index)
"""
# Модель с явным контролем типов
class TypeSafeGCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim).to(device)
        self.conv2 = GCNConv(hidden_dim, output_dim).to(device)

        # Принудительная установка типа параметров
        for param in self.parameters():
            param.data = param.data.float()

    def forward(self, x, edge_index):
        x = x.float()  # Явное преобразование типа
        x = F.relu(self.conv1(x, edge_index))
        return self.conv2(x, edge_index)"""

'\n# Модель с явным контролем типов\nclass TypeSafeGCN(nn.Module):\n    def __init__(self, input_dim, hidden_dim, output_dim):\n        super().__init__()\n        self.conv1 = GCNConv(input_dim, hidden_dim).to(device)\n        self.conv2 = GCNConv(hidden_dim, output_dim).to(device)\n\n        # Принудительная установка типа параметров\n        for param in self.parameters():\n            param.data = param.data.float()\n\n    def forward(self, x, edge_index):\n        x = x.float()  # Явное преобразование типа\n        x = F.relu(self.conv1(x, edge_index))\n        return self.conv2(x, edge_index)'

In [ ]:
# Инициализация
dataset = TypeSafeMusicDataset(data)
loader = DataLoader(dataset, batch_size=1)

model = TypeSafeGCN(
    input_dim=len(data.columns)-3,
    hidden_dim=64,
    output_dim=32).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
# Функция обучения с контролем типов
def train_safe():
    model.train()
    for epoch in range(30):
        optimizer.zero_grad()

        for batch in loader:
            batch = batch.to(device)

            # Явное преобразование типов
            x = batch.x.float()
            edge_index = batch.edge_index.long()

            # Forward pass
            out = model(x, edge_index)

            # Простой loss для демонстрации
            target = torch.rand_like(out).float().to(device)
            loss = F.mse_loss(out, target)

            # Backward pass
            loss.backward()
            optimizer.step()

            # Очистка памяти
            del batch, x, edge_index, out, target
            torch.cuda.empty_cache()
            gc.collect()

        print(f'Epoch {epoch+1}/30, Loss: {loss.item():.4f}')

In [ ]:
# Запуск обучения
train_safe()

Epoch 1/30, Loss: 0.0978
Epoch 2/30, Loss: 0.0971
Epoch 3/30, Loss: 0.0965
Epoch 4/30, Loss: 0.0960
Epoch 5/30, Loss: 0.0956
Epoch 6/30, Loss: 0.0950
Epoch 7/30, Loss: 0.0946
Epoch 8/30, Loss: 0.0941
Epoch 9/30, Loss: 0.0937
Epoch 10/30, Loss: 0.0933
Epoch 11/30, Loss: 0.0931
Epoch 12/30, Loss: 0.0927
Epoch 13/30, Loss: 0.0923
Epoch 14/30, Loss: 0.0921
Epoch 15/30, Loss: 0.0919
Epoch 16/30, Loss: 0.0916
Epoch 17/30, Loss: 0.0914
Epoch 18/30, Loss: 0.0912
Epoch 19/30, Loss: 0.0910
Epoch 20/30, Loss: 0.0909
Epoch 21/30, Loss: 0.0907
Epoch 22/30, Loss: 0.0904
Epoch 23/30, Loss: 0.0903
Epoch 24/30, Loss: 0.0902
Epoch 25/30, Loss: 0.0899
Epoch 26/30, Loss: 0.0897
Epoch 27/30, Loss: 0.0896
Epoch 28/30, Loss: 0.0896
Epoch 29/30, Loss: 0.0894
Epoch 30/30, Loss: 0.0892


In [ ]:
# Функция рекомендаций
def safe_recommend(song_list, n=5):
    # Поиск индексов
    indices = []
    for song in song_list:
        match = data[data['name'] == song['name']]
        if not match.empty:
            indices.append(match.index[0])
        else:
            print(f"Song '{song['name']}' not found")
            return []

    # Получение эмбеддингов
    with torch.no_grad():
        model.eval()
        embeddings = model(dataset.node_features.float(), dataset.edge_index)

    # Расчет схожести
    input_emb = embeddings[indices].mean(dim=0)
    cos_sim = F.cosine_similarity(input_emb.unsqueeze(0), embeddings)

    # Фильтрация результатов
    top_indices = cos_sim.topk(n + len(indices))[1].cpu().numpy()
    recs = data.iloc[top_indices]
    recs = recs[~recs['name'].isin([s['name'] for s in song_list])]

    return recs.head(n)[['name', 'artists', 'year']]

In [ ]:
# Тестовый пример
recommendations = safe_recommend([{'name': 'Break Stuff', 'artists': 'Limp Bizkit'}], n=5)

print("\nRecommendations:")
print(recommendations)


Recommendations:
                                                     name  \
13608                                      Man in the Box   
140743  the song they played (when i crashed into the ...   
17822                                    The Show Goes On   
14604                                          Range Life   

                          artists  year  year  
13608         ['Alice In Chains']  1990  1990  
140743  ['Lil Peep', 'Lil Tracy']  2020  2020  
17822             ['Lupe Fiasco']  2011  2011  
14604                ['Pavement']  1994  1994  


In [ ]:
def save_full_model(model, scaler, feature_columns, filename='music_recommender.h5'):
    with h5py.File(filename, 'w') as f:
        # Сохраняем параметры модели
        model_grp = f.create_group('model')
        model_grp.attrs['input_dim'] = model.input_dim
        model_grp.attrs['hidden_dim'] = model.hidden_dim
        model_grp.attrs['output_dim'] = model.output_dim

        # Сохраняем веса
        weights_grp = model_grp.create_group('weights')
        for name, param in model.named_parameters():
            weights_grp.create_dataset(name, data=param.cpu().detach().numpy())

        # Сохраняем StandardScaler
        scaler_grp = f.create_group('scaler')
        scaler_grp.create_dataset('mean_', data=scaler.mean_)
        scaler_grp.create_dataset('scale_', data=scaler.scale_)
        scaler_grp.create_dataset('var_', data=scaler.var_)

        # Сохраняем feature columns
        f.create_dataset('feature_columns',
                        data=np.array(feature_columns, dtype='S'),
                        dtype=h5py.string_dtype())

In [ ]:
def load_full_model(filename, device):
    with h5py.File(filename, 'r') as f:
        # Проверка существования групп
        if 'model' not in f:
            raise ValueError("Invalid model file: 'model' group missing")

        # Загрузка параметров модели
        model_grp = f['model']
        input_dim = model_grp.attrs['input_dim']
        hidden_dim = model_grp.attrs['hidden_dim']
        output_dim = model_grp.attrs['output_dim']

        # Инициализация модели
        model = TypeSafeGCN(input_dim, hidden_dim, output_dim).to(device)

        # Загрузка весов
        weights_grp = model_grp['weights']
        for name, param in model.named_parameters():
            param.data = torch.tensor(weights_grp[name][...], device=device)

        # Загрузка scaler
        scaler = StandardScaler()
        scaler.mean_ = f['scaler/mean_'][...]
        scaler.scale_ = f['scaler/scale_'][...]
        scaler.var_ = f['scaler/var_'][...]

        # Загрузка feature columns
        feature_columns = [col.decode('utf-8') for col in f['feature_columns'][...]]

    return model, scaler, feature_columns

In [ ]:
# тест сохранения и загрузки

scaler = StandardScaler()
scaler.fit(data.iloc[:, 3:].values)
feature_columns = data.columns[3:].tolist()

save_full_model(model, scaler, feature_columns, 'fixed_model2.h5')

# Загрузка модели
loaded_model, loaded_scaler, loaded_features = load_full_model('fixed_model2.h5', device)

# Проверка атрибутов
print(f"Input dim: {loaded_model.input_dim}")
print(f"Hidden dim: {loaded_model.hidden_dim}")
print(f"Output dim: {loaded_model.output_dim}")

Input dim: 14
Hidden dim: 64
Output dim: 32


In [ ]:
# Загрузка модели
loaded_model, loaded_scaler, loaded_features = load_full_model('fixed_model2.h5', device)

In [ ]:
loaded_model, _, _ = load_full_model('/content/fixed_model2.h5', device)